In [1]:
import pandas as pd

In [2]:
# Load the datasets

teams = pd.read_csv('data/teams.csv').convert_dtypes()
results = pd.read_csv('data/results.csv').convert_dtypes()
fixtures = pd.read_csv('data/fixtures.csv').convert_dtypes()
players = pd.read_csv('data/players.csv').convert_dtypes()
startingXI = pd.read_csv('data/startingXI.csv').convert_dtypes()
odds = pd.read_csv('data/odds.csv').convert_dtypes()

datasets = [teams, results, fixtures, players, startingXI, odds]

In [3]:
# inspect values and if any values are missing 
for dataset in datasets:
  print(dataset.head())
  print(dataset.info())
  print('-------------')

    TeamName  TeamID
0  Arlington       1
1    Anaheim       2
2    Atlanta       3
3  Baltimore       4
4     Boston       5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TeamName  28 non-null     string
 1   TeamID    28 non-null     Int64 
dtypes: Int64(1), string(1)
memory usage: 608.0 bytes
None
-------------
   SeasonID  Gameweek  MatchID  HomeTeamID  HomeScore  HomeShots  AwayTeamID  \
0         1         1        1           7          1         17           1   
1         1         1        2           6          1          8           8   
2         1         1        3           5          5         21           9   
3         1         1        4           4          2         25          10   
4         1         1        5           3          3         13          11   

   AwayScore  AwayShots  
0          1         12  
1          3  

In [4]:
results['SeasonID'].value_counts()

SeasonID
1    756
2    756
Name: count, dtype: Int64

Input data summary:
* No null or missing values, correct data types 
* Each table and column names are self explanatory
* Dataset provided for season 1 & season 2 

# Exploring the First Season

This league uses the same rules for determining the order of teams as the English Premier League.


### 1. Which team won the league in the first season?


In [5]:
# Merge results with teams 
results_with_teams = results.merge(teams, left_on='HomeTeamID', right_on='TeamID').merge(teams, left_on='AwayTeamID', right_on='TeamID', suffixes=('_Home', '_Away'))

# Calculate match points: win 3 , draw 1 , loss 0
results_with_teams['HomePoints'] = results_with_teams.apply(lambda x: 3 if x['HomeScore'] > x['AwayScore'] else (1 if x['HomeScore'] == x['AwayScore'] else 0), axis=1)
results_with_teams['AwayPoints'] = results_with_teams.apply(lambda x: 3 if x['HomeScore'] < x['AwayScore'] else (1 if x['HomeScore'] == x['AwayScore'] else 0), axis=1)

# pick only season 1 
results_with_teams_s1 = results_with_teams[results_with_teams['SeasonID'] == 1]

# add up both home and away points, get the team name as well  
home_points = results_with_teams_s1.groupby('HomeTeamID').agg(HomePoints = pd.NamedAgg(column='HomePoints', aggfunc='sum'),
                                                              HomeTeamName = pd.NamedAgg(column='TeamName_Home', aggfunc='first'),
                                                              AwayTeamName = pd.NamedAgg(column='TeamName_Away', aggfunc='first')).reset_index()
away_points = results_with_teams_s1.groupby('AwayTeamID')['AwayPoints'].sum().reset_index()

# add up  the home and away points 
total_points = home_points.merge(away_points, left_on='HomeTeamID', right_on='AwayTeamID')
total_points['TotalPoints'] = total_points['HomePoints'] + total_points['AwayPoints']

In [6]:
# show the full ranking and get the winning the team id, name
ranking_s1 = total_points.sort_values(by='TotalPoints', ascending=False)[['HomeTeamID', 'HomeTeamName', 'TotalPoints']].reset_index(drop=True)
winning_team = total_points[total_points['TotalPoints'] == total_points['TotalPoints'].max()][['HomeTeamID', 'HomeTeamName']].reset_index(drop=True).values[0]
win_team_id, win_team_name = tuple(winning_team)

In [7]:
# Season 1 teams ranked by points
ranking_s1

,HomeTeamID,HomeTeamName,TotalPoints
0,15,Miami,138
1,8,Cincinnati,125
2,4,Baltimore,117
3,19,New York S,113
4,5,Boston,106
5,27,Seattle,105
6,6,Chicago B,105
7,21,Oakland,96
8,7,Chicago H,95
9,24,St. Louis,94


In [8]:
print(f'Winning team ID: {win_team_id}, Winning team Name: {win_team_name}')

Winning team ID: 15, Winning team Name: Miami


`Results:` The winning team is Miami with 138 points in the first season. With Cinccinnati and Baltimore securing the 2nd and 3rd position consecutively. 

### 2. At what point in the season did that team secure their league title?


In [9]:

# calculate cumulative points for each team by week
cumulative_points = results_with_teams_s1.copy()
# sort by gameweek and match ID 
cumulative_points = cumulative_points.sort_values(by=['Gameweek', 'MatchID']).reset_index(drop=True)
# track Miami points for each week
cumulative_points['MiamiPoints'] = 0

for i, row in cumulative_points.iterrows():
  if row['HomeTeamID'] == win_team_id:
    cumulative_points.at[i, 'MiamiPoints'] = cumulative_points.at[i-1, 'MiamiPoints'] + row['HomePoints'] if i > 0 else row['HomePoints']
  elif row['AwayTeamID'] == win_team_id:
    cumulative_points.at[i, 'MiamiPoints'] = cumulative_points.at[i-1, 'MiamiPoints'] + row['AwayPoints'] if i > 0 else row['AwayPoints']
  else:
    cumulative_points.at[i, 'MiamiPoints'] = cumulative_points.at[i-1, 'MiamiPoints'] if i > 0 else 0
  
  # get the maximum points for other teams 
  max_points_other_teams = total_points[total_points['HomeTeamID'] != win_team_id]['TotalPoints'].max()
  
  # check the week Miami surpress that that 
  win_gameweek = cumulative_points[cumulative_points['MiamiPoints'] > max_points_other_teams]['Gameweek'].min()
  match_id = cumulative_points[cumulative_points['MiamiPoints'] > max_points_other_teams]['MatchID'].min()

In [10]:
print(f'Miami secured the win at week {win_gameweek} of season 1, at MatchID: {match_id}')

Miami secured the win at week 50 of season 1, at MatchID: 699


### 3. What result was the biggest upset?

In [11]:
# To compare the biggest upset compare the match results with bookmaker odds 

#merge results with odds 
results_with_odds = results_with_teams_s1.merge(odds, on='MatchID')

# determine the expected outcome of a match based on the bookermaker odds 
results_with_odds['ExpectedOutcome'] = results_with_odds.apply(lambda row: 'HomeWin' if row['Home'] < row['Draw'] and row['Home'] < row['Away'] else ('Draw' if row['Draw'] < row['Home'] and row['Draw'] < row['Away'] else 'AwayWin'), axis=1)

# determine the actual outcome of each match 
results_with_odds['ActualOutcome'] = results_with_odds.apply(lambda row: 'HomeWin' if row['HomeScore'] > row['AwayScore'] else ('Draw' if row['HomeScore'] == row['AwayScore'] else 'AwayWin'), axis=1)

# # identify the cases where expected outcome is different from actual outcome and note the odds
# results_with_odds['UpsetOdds'] = results_with_odds.apply(lambda row: row['Home'] if row['ActualOutcome'] == 'AwayWin' else (row['Away'] if row['ActualOutcome'] == 'HomeWin' else row['Draw']), axis=1)



In [12]:
results_with_odds[['MatchID', 'HomeTeamID', 'AwayTeamID', 'HomeScore', 'AwayScore', 'Home', 'Draw', 'Away', 'ExpectedOutcome', 'ActualOutcome']].head()

,MatchID,HomeTeamID,AwayTeamID,HomeScore,AwayScore,Home,Draw,Away,ExpectedOutcome,ActualOutcome
0,1,7,1,1,1,1.92,3.96,3.89,HomeWin,Draw
1,2,6,8,1,3,2.16,3.88,3.23,HomeWin,AwayWin
2,3,5,9,5,1,1.21,7.65,13.17,HomeWin,HomeWin
3,4,4,10,2,0,1.05,17.0,43.49,HomeWin,HomeWin
4,5,3,11,3,3,3.12,3.6,2.32,AwayWin,Draw


In [23]:
# find the matches where expected outcome is not the same as actual outcome 
upsets = results_with_odds[results_with_odds['ExpectedOutcome'] != results_with_odds['ActualOutcome']].reset_index(drop=True)
# get the magnitude of upsets 
upsets['UpsetOdds'] = upsets.apply(lambda row: row['Home'] if row['ActualOutcome'] == 'AwayWin' else (row['Away'] if row['ActualOutcome'] == 'HomeWin' else row['Draw']), axis=1)
biggest_upset = upsets.loc[upsets['UpsetOdds'].idxmax()]

In [26]:
print(f'Biggest upset: \n{biggest_upset}')

Biggest upset: 
SeasonID                   1
Gameweek                  12
MatchID                  168
HomeTeamID                15
HomeScore                  3
HomeShots                 38
AwayTeamID                16
AwayScore                  3
AwayShots                 10
TeamName_Home          Miami
TeamID_Home               15
TeamName_Away      Milwaukee
TeamID_Away               16
HomePoints                 1
AwayPoints                 1
Home                    1.03
Draw                   23.35
Away                   75.22
ExpectedOutcome      HomeWin
ActualOutcome           Draw
UpsetOdds              23.35
Name: 74, dtype: object
